Выполнить Стэкинг, Бэгин, Вотинг и Бустинг. При реализации алгоритмов не использовать готовые решения. 
За сровнение взять CatBoostClassifier как базовая метрика качества. Сравнить результат с реализацией своих ансамблей. 
Для однозначности и интерпретируемости результатов использовать приложенный набор данных. 
При реализации бустинга - просто сокращайте набор данных на котором модель отработала хорошо (правильно предсказанные данные). 

In [34]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score  
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier


np.set_printoptions(edgeitems=20, linewidth=170000, 
    formatter=dict(float=lambda x: "%.5g" % x))
#date_current = datetime.strptime("26-02-2010", '%d-%m-%Y').timetuple().tm_yday
with open('winequality-white.csv', newline='', encoding='Windows-1251') as f:
    str = f.read().replace(",", "." ) 
    str = str.replace(";",",")    
    with open('wine.csv','w',  encoding='Windows-1251') as file2:    
        file2.write(str)
df = pd.read_csv("wine.csv",encoding='Windows-1251' ,on_bad_lines='skip',
                    
                    )

df
y = df['quality']
X = df.drop(['quality'] ,axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=72)

n_estimators=160
max_depth =14
min_samples_split = 2
R_f_classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                         min_samples_split=min_samples_split, random_state=42)
R_f_classifier.fit(X_train,y_train)
y_pred_RF_forest = R_f_classifier.predict(X_test)
print(f"The r2 score of the model RFClassifier is: {r2_score(y_test, y_pred_RF_forest)}")


svc_clf = SVC(gamma = "auto", probability = True, random_state=42)
svc_clf.fit(X_train,y_train)
y_pred = svc_clf.predict(X_test)    
print(" SVC accuracy by SVC",accuracy_score(y_test,y_pred))

# Бэггинг классификатор.
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42),n_estimators=40, max_samples=100,bootstrap=True,n_jobs=-1,random_state = 42)
    #SVC(gamma = "auto", probability = True, random_state=42), n_estimators=40, max_samples=100,bootstrap=True,n_jobs=-1,random_state = 42)
bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)    
print(" Bagging accuracy by DesisionTree",accuracy_score(y_test,y_pred))


The r2 score of the model RFClassifier is: 0.4535102524073592
 SVC accuracy by SVC 0.5653061224489796
 Bagging accuracy by DesisionTree 0.5510204081632653


Основной целью здесь была Бэггинг классификация. Она дала значение 0.55.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
clf1 = KNeighborsClassifier(n_neighbors=15)
clf2 = RandomForestClassifier(n_estimators=130, random_state=1)
clf3 = GaussianNB()
clf4 = DecisionTreeClassifier(random_state=42)
clf5 = GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),

voting_clf = VotingClassifier(estimators=[('rf', clf2),('lr', clf1),  ('gnb', clf3),('dtc', clf4)], voting='soft')

voting_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)    
print(" Voting accuracy 1",accuracy_score(y_test,y_pred))


 Voting accuracy 1 0.5571428571428572


Вотинг классификатор показывает значение 0.55. Он не дает прироста по отношению к лучшему из классификаторов.

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import root_mean_squared_error

estimators = [
    ('rf', RandomForestClassifier(n_estimators=29, random_state=42)),    
    #('svr', GaussianNB())
    ('svr', LinearSVC(random_state=22))
]
clf = StackingClassifier(
    estimators=estimators #, final_estimator=LogisticRegression()
)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(" Stacking accuracy 1",accuracy_score(y_test,y_pred)," RMSE", root_mean_squared_error(y_test,y_pred))

 Stacking accuracy 1 0.6989795918367347  RMSE 0.6380774695464941


d:\Soft\ProgramFiles\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Стэкинг дает наилучший результат, почти 0.7

In [33]:
clf_cat_boost = CatBoostClassifier(iterations=150,
                           
                           devices='1')
clf_cat_boost.fit(X_train, y_train)
y_pred = clf_cat_boost.predict(X_test)
print(" Stacking accuracy 1",accuracy_score(y_test,y_pred)," RMSE", root_mean_squared_error(y_test,y_pred))



Learning rate set to 0.421476
0:	learn: 1.4430048	total: 8.22ms	remaining: 1.23s
1:	learn: 1.2998414	total: 14.6ms	remaining: 1.08s
2:	learn: 1.2072460	total: 23.7ms	remaining: 1.16s
3:	learn: 1.1544309	total: 30.6ms	remaining: 1.12s
4:	learn: 1.1141332	total: 37.6ms	remaining: 1.09s
5:	learn: 1.0808320	total: 44.4ms	remaining: 1.06s
6:	learn: 1.0533901	total: 50.6ms	remaining: 1.03s
7:	learn: 1.0385460	total: 56.7ms	remaining: 1.01s
8:	learn: 1.0226363	total: 64.3ms	remaining: 1.01s
9:	learn: 1.0090577	total: 70.1ms	remaining: 981ms
10:	learn: 0.9987669	total: 75.7ms	remaining: 957ms
11:	learn: 0.9900048	total: 80.6ms	remaining: 927ms
12:	learn: 0.9857540	total: 85.7ms	remaining: 904ms
13:	learn: 0.9733581	total: 91.2ms	remaining: 886ms
14:	learn: 0.9648146	total: 96.2ms	remaining: 865ms
15:	learn: 0.9530299	total: 101ms	remaining: 849ms
16:	learn: 0.9425972	total: 106ms	remaining: 829ms
17:	learn: 0.9310650	total: 111ms	remaining: 810ms
18:	learn: 0.9235319	total: 115ms	remaining: 79

Кэтбуст дает результат 0.65. 